In [ ]:
import numpy
import pandas
import math
from datetime import datetime
import pandas_datareader.data as web
import xlrd

In [ ]:
%load_ext autoreload
%autoreload 2
import bond_simulator

In [ ]:
import seaborn
from matplotlib import pyplot as plt
seaborn.set(style='whitegrid')
seaborn.set_context('poster')

# OECD Short

These are 3-month rates.

In [ ]:
oecd_short = pandas.read_excel('OECD IMF RoE Interest Rates.xlsx',
                          usecols='A,F,G',
                          parse_dates=['TIME'],
                          sheet_name='DP_LIVE_ShortM',
                          converters = {'Value': lambda x: x/100}
                         )

In [ ]:
def get_country(country, df):
    cc = df[df['LOCATION'] == country].sort_values('TIME')
    return pandas.Series(data=cc['Value'].values, index=cc['TIME'])
get_country('USA', oecd_short).head()

# OECD Long

These are 10-year rates.

In [ ]:
oecd_long = pandas.read_excel('OECD IMF RoE Interest Rates.xlsx',
                          usecols='A,F,G',
                          parse_dates=['TIME'],
                          sheet_name='DP_LIVE_LongM',
                          converters = {'Value': lambda x: x/100}
                         )

In [ ]:
get_country('USA', oecd_long).head()

# IMF Rates

This sheet contains "bond", "bill", and "money market" rates. We need to do some extra processing here to get everything in the right shape for us to use.

* "bond" maturity is ???
* "bill" maturity is anywhere from 3- to 12-months depending on the country
* "money market" maturity is (presumably?) overnight or 1-month?

In [ ]:
imf = pandas.read_excel('OECD IMF RoE Interest Rates.xlsx',
                        sheet_name='IMF Monthly',
                        usecols='A,F,H:AKF', # Warning! AKF goes to January 2020 only.
                        skipfooter=18
                         )
imf.head()

In [ ]:
imf = imf.rename(columns={'Unnamed: 0': 'Country'})

def slice_imf(df, indicator_code):
    df_slice = df[df['Indicator Code'] == indicator_code]
    dates = [datetime.strptime(x, '%YM%m') for x in df_slice.T.index[2:]]
    n = pandas.DataFrame(columns=df_slice['Country'], data=df_slice.T[2:].values, index=dates)
    return n.applymap(lambda x: x / 100)

imf_bonds = slice_imf(imf, 'FIGB_PA')
imf_moneymarket = slice_imf(imf, 'FIMM_PA')
imf_bills = slice_imf(imf, 'FITB_PA')
imf_moneymarket.dropna().head()

In [ ]:
def make_series(country):
    short = imf_bills[country].combine_first(get_country(country, oecd_short)).dropna()
    long = imf_bonds[country].combine_first(get_country(country, oecd_long)).dropna()

    df = pandas.DataFrame(columns=range(1,11), data={1: short, 10: long}, dtype=numpy.float64)
    df = df.fillna(method='backfill', axis=0) # fill UP along a column, guarantees we have some kind of yield curve
    #df = df.dropna(thresh=2) # we might only have 1 rate (i.e. only short or only long; skip those years)
    df.interpolate(axis=1, inplace=True)
    #df.fillna(method='backfill', axis=1, inplace=True)
    return df

Prefer IMF rates. Are there any cases where we have OECD rates but not IMF rates?

In [ ]:
make_series('USA')['1953':]

In [ ]:
# given a series of yearly columns, convert it to monthly columns
def explode_series(annual_series):
    years = len(annual_series.columns)
    df = pandas.DataFrame(index=annual_series.index, columns=range(1, 12 * years + 1), dtype=numpy.float64)
    for index, cols in annual_series.iterrows():
        for year, rate in cols.iteritems():
            df.loc[index][year * 12] = rate
    df.interpolate(axis=1, inplace=True)
    return df.fillna(method='backfill', axis=1)

# we need to change the yields from annual yields to monthly yields
def make_yield_monthly(df):
    return df.applymap(lambda x: math.pow(1+x, 1/12) - 1)

In [ ]:
country = 'USA'
short = imf_bills[country].combine_first(get_country(country, oecd_short)).dropna()
long = imf_bonds[country].combine_first(get_country(country, oecd_long)).dropna()
df = pandas.DataFrame(columns=range(1,11), data={1: short, 10: long}, dtype=numpy.float64)
#df = df.fillna(method='backfill', axis=0)
df.interpolate(axis=1, inplace=True)
df.fillna(method='backfill', axis=1)
df.head()

# debug what is wrong with USA, GBR, CAN

In [ ]:
country_name = 'USA'
annual_yields = make_series(country_name)
exploded_yields = explode_series(annual_yields)
monthly_yields = make_yield_monthly(exploded_yields)
country_m = bond_simulator.simulate_turnover((12*10), (12*4), monthly_yields)
country_m.to_csv(f'{country_name.lower()}-monthly.csv')
country_m.head()

In [ ]:
# aggregate monthly changes to an annual change
# or...could just compare NAV via a shift.
s = country_m['Change'] + 1
g = s.groupby(numpy.arange(len(s))//12, axis=0).prod()
g = g - 1
g.index = pandas.date_range(country_m.index[0], periods=len(g), freq='AS-JAN')
g

# Annual turnover

In [ ]:
country_name = 'USA'
annual = make_series(country_name)[::12]
country_a = bond_simulator.simulate_turnover(10, 4, annual)
country_a.to_csv(f'{country_name.lower()}-annual.csv')
country_a.head()

In [ ]:
def plot_all(key, monthly, annual, title=None):
    g = monthly[key].groupby(numpy.arange(len(s))//12, axis=0).mean()
    g.index = pandas.date_range(monthly.index[0], periods=len(g), freq='AS-JAN')
    
    if not title:
        title = key

    plt.figure(figsize=(11,9))
    plt.title(title)
    seaborn.lineplot(data=pandas.DataFrame(data={
        'Annual': annual[key],
        'Monthly (January)': monthly[key][::12],
        'Monthly (Average)': g
    }))   

In [ ]:
plot_all('Maturity', country_m, country_a)

In [ ]:
plot_all('Coupon', country_m, country_a)

In [ ]:
plot_all('YTM', country_m, country_a, title='Yield to Maturity')

In [ ]:
plot_all('Duration', country_m, country_a)